In [175]:
import sqlite3
import numpy as np
import pandas as pd
from jdcal import gcal2jd, jd2gcal, jcal2jd, MJD_0
import keras
import math
import datetime
import tensorflowjs as tfjs
import pickle
import julian as julian
import datetime
import calendar
import sklearn as metrics
from keras.models import Sequential, model_from_yaml, load_model
from keras.layers import Dense, LSTM, Dropout, Embedding, Masking
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [181]:
def convertJulianDateToDays(y, jd):
    month = 1
    day = 0
    while jd - calendar.monthrange(y,month)[1] > 0 and month <= 12:
        jd = jd - calendar.monthrange(y,month)[1]
        month = month + 1
    return datetime.datetime(y, month, jd)

def convertJulianTimeToCategory(param):
    if (param[0] == None):
        return "n_a";
        
    row = int(param[0])
    if row >= 0 and row < 400:
        return 'early_morning'
    elif row >= 400 and  row < 800:
        return 'mid_morning'
    elif row >= 800 and  row < 1200:
        return 'late_morning'
    elif row >=1200 and  row <1600:
        return 'afternoon'
    elif row >=1600 and  row <2000:
        return 'evening'
    elif row >=2000 and  row <2400:
        return 'night'
    else:
        return 'n_a'

def encodeOutputVariable(y):
    labelencoder_Y_Origin = LabelEncoder()
    y = labelencoder_Y_Origin.fit_transform(y.astype(str))
    return y

def encodeCategoricalData(X, index):
    # encode categorical data
    labelencoder_X_Origin = LabelEncoder()
    X[:, index] = labelencoder_X_Origin.fit_transform(X[:, index].astype(str))
    return X    

def manualEncodeLongStrings(X, column):
    index = 0
    test = 0
    keys = {}
    for row in X:
        key = row[column].replace(", ", "").replace(" ", "")
        if (keys.get(key) == None):
            keys[key] = index
            index += 1
        X[test][column] = keys.get(key)
        test += 1
    return X
    
def defaultMinimumValues(values): 
    for index, x in enumerate(values):
        x = [float(y) if y != None and float(y) > 0.0 else 0 for y in x]
        values[index] = x
    return pd.DataFrame(values)

def convertDateColumns(X, column):
    X = X[:, column].apply(lambda row: julian.from_jd(row, fmt='mmddyyyy'))
    print(X[:, column])
    return X.values

def encodeHotEncoder(X, categoryIndex):
    # meant to create dummy variables for each category data
    # you only use it for one column at a time, output will be the number of columns
    # needed to represent all discrete values of column
    onehotencoder = OneHotEncoder(categorical_features = [categoryIndex])
    X = defaultMinimumValues(X)
    X = onehotencoder.fit_transform(X.astype(str)).toarray()    
    X = X[:, 1:]
    return X

def determineTotalTime(startDay, startTime, endDay, endTime):
    minsPerDay = 24 * 60
    totalStart = (int(startDay) * minsPerDay) + int(startTime)
    totalEnd = (int(endDay) * minsPerDay) + int(endTime)
    return totalEnd - totalStart

def createArtificialNeuralNetwork(X_train, y_train):
    # create ANN

    # initialize the ann
    classifier = Sequential()

    # adding the input layer and the first hidden layer
    classifier.add(Dense(100, kernel_initializer = "uniform", activation = "relu", input_dim = 41))
    
    classifier.add(Dense(90, kernel_initializer = "uniform", activation="relu"))
    
    classifier.add(Dense(75, kernel_initializer = "uniform", activation="relu"))
    
    # adding the second hidden layer
    classifier.add(Dense(50, kernel_initializer = "uniform", activation = "relu"))

    # adding the third hidden layer
    classifier.add(Dense(25, kernel_initializer = "uniform", activation = "relu"))

    classifier.add(Dense(20, kernel_initializer = "uniform", activation="relu"))
    
    # adding the fourth hidden layer
    classifier.add(Dense(10, kernel_initializer = "uniform", activation = "relu"))
    
    classifier.add(Dense(5, kernel_initializer = "uniform", activation="relu"))

    # adding the output layer 
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))

    # compiling the ANN
    classifier.compile(optimizer = "adam", loss = "mean_squared_error", metrics = ["accuracy"])

    # fitting the ANN to the training set
    classifier.fit(X_train, y_train, batch_size = 100, epochs = 200)
    
def creatingRandomForestPredictor(X_train, y_train, X_test, y_test):
    print("\nRandom Forest Classifier Section")
    print("---------------------------------")
    
    # initialize the Random Forest Classifier
    random_forest_classifier = RandomForestClassifier(**{'n_jobs': -1,
        'n_estimators': 1500,
        'warm_start': True, 
        'max_features': 0.5,
        'max_depth': 15,
        'min_samples_leaf': 2,
        'max_features' : 'sqrt',
        'random_state' : 0,
        'verbose': 0
                                                      })
    
    oversampler = SMOTE(random_state=0)
    smote_X_train, smote_y_train = oversampler.fit_sample(X_train, y_train)
    
    # fitting Random Forest to the training set
    random_forest_classifier.fit(smote_X_train, smote_y_train)
    
    # Predicting the Test set results
    rf_y_pred = random_forest_classifier.predict(X_test)
    
    # use the threshold of error to determine whether a prediction is valid
    rf_y_pred = (rf_y_pred > 0.5)
    
    # making the confusion matrix
    #cm = metrics.confusion_matrix(y_test.ravel(), rf_y_pred.ravel())
    
    print("Training set Score: ", random_forest_classifier.score(X_train, y_train))
    print("Testing set Score: ", random_forest_classifier.score(X_test, y_test)) 
    
def createNaiveBayesModel(X_train, y_train, X_test, y_test):
    classifier = GaussianNB()
    
    classifier.fit(X_train, y_train)
    
    y_pred = classifier.predict(X_test)
    
    y_pred = (y_pred > 0.5)
    
    print("Training set Score: ", classifier.score(X_train, y_train))
    print("Testing set Score: ", classifier.score(X_test, y_test))    

In [161]:
sqlite_file = "./wildfires.sqlite"

# connecting to the database file and saving the select
conn = sqlite3.connect(sqlite_file)
dataset = pd.read_sql_query("select * from Fires limit 5000;", conn)

In [162]:
dataset.iloc[:,:]

OBJECTID  FOD_ID      FPA_ID SOURCE_SYSTEM_TYPE SOURCE_SYSTEM  \
0            1       1  FS-1418826                FED   FS-FIRESTAT   
1            2       2  FS-1418827                FED   FS-FIRESTAT   
2            3       3  FS-1418835                FED   FS-FIRESTAT   
3            4       4  FS-1418845                FED   FS-FIRESTAT   
4            5       5  FS-1418847                FED   FS-FIRESTAT   
5            6       6  FS-1418849                FED   FS-FIRESTAT   
6            7       7  FS-1418851                FED   FS-FIRESTAT   
7            8       8  FS-1418854                FED   FS-FIRESTAT   
8            9       9  FS-1418856                FED   FS-FIRESTAT   
9           10      10  FS-1418859                FED   FS-FIRESTAT   
10          11      11  FS-1418861                FED   FS-FIRESTAT   
11          12      12  FS-1418863                FED   FS-FIRESTAT   
12          13      13  FS-1418865                FED   FS-FIRESTAT   
13          14      14  FS-1418872                FED   FS-FIRESTAT   
14          15      15  FS-1418874                FED   FS-FIRESTAT   
15          16      16  FS-1418876                FED   FS-FIRESTAT   
16          17      17  FS-1418878                FED   FS-FIRESTAT   
17          18      18  FS-1418881                FED   FS-FIRESTAT   
18          19      19  FS-1418884                FED   FS-FIRESTAT   
19          20      20  FS-1418887                FED   FS-FIRESTAT   
20          21      21  FS-1418888                FED   FS-FIRESTAT   
21          22      22  FS-1418893                FED   FS-FIRESTAT   
22          23      23  FS-1418894                FED   FS-FIRESTAT   
23          24      24  FS-1418895                FED   FS-FIRESTAT   
24          25      25  FS-1418896                FED   FS-FIRESTAT   
25          26      26  FS-1418897                FED   FS-FIRESTAT   
26          27      27  FS-1418898                FED   FS-FIRESTAT   
27          28      28  FS-1418903                FED   FS-FIRESTAT   
28          29      29  FS-1418904                FED   FS-FIRESTAT   
29          30      30  FS-1418905                FED   FS-FIRESTAT   
...        ...     ...         ...                ...           ...   
4970      4971    4994  FS-1428564                FED   FS-FIRESTAT   
4971      4972    4995  FS-1428565                FED   FS-FIRESTAT   
4972      4973    4996  FS-1428566                FED   FS-FIRESTAT   
4973      4974    4997  FS-1428567                FED   FS-FIRESTAT   
4974      4975    4998  FS-1428575                FED   FS-FIRESTAT   
4975      4976    4999  FS-1428576                FED   FS-FIRESTAT   
4976      4977    5000  FS-1428577                FED   FS-FIRESTAT   
4977      4978    5001  FS-1428580                FED   FS-FIRESTAT   
4978      4979    5002  FS-1428585                FED   FS-FIRESTAT   
4979      4980    5003  FS-1428586                FED   FS-FIRESTAT   
4980      4981    5004  FS-1428589                FED   FS-FIRESTAT   
4981      4982    5005  FS-1428598                FED   FS-FIRESTAT   
4982      4983    5006  FS-1428599                FED   FS-FIRESTAT   
4983      4984    5007  FS-1428601                FED   FS-FIRESTAT   
4984      4985    5008  FS-1428603                FED   FS-FIRESTAT   
4985      4986    5009  FS-1428610                FED   FS-FIRESTAT   
4986      4987    5010  FS-1428611                FED   FS-FIRESTAT   
4987      4988    5011  FS-1428612                FED   FS-FIRESTAT   
4988      4989    5012  FS-1428613                FED   FS-FIRESTAT   
4989      4990    5013  FS-1428614                FED   FS-FIRESTAT   
4990      4991    5014  FS-1428616                FED   FS-FIRESTAT   
4991      4992    5015  FS-1428617                FED   FS-FIRESTAT   
4992      4993    5016  FS-1428619                FED   FS-FIRESTAT   
4993      4994    5017  FS-1428620                FED   FS-FIRESTAT   
4994      4995    5018  FS-142

In [163]:
# split dataset into train and test lists
X = dataset.iloc[:, [34, 35, 30, 31, 29, 28, 19, 20, 22, 25, 27]].values
y = dataset.iloc[:, 23].values
X

array([['CA', '63', 40.03694444, ..., '1300', 2453403.5, '1730'],
       ['CA', '61', 38.93305556, ..., '0845', 2453137.5, '1530'],
       ['CA', '17', 38.98416667, ..., '1921', 2453156.5, '2024'],
       ...,
       ['WA', '65', 48.39083333, ..., '1030', 2453583.5, '1500'],
       ['AZ', '13', 33.98638889, ..., '0842', 2453686.5, '1015'],
       ['WA', '65', 48.35694444, ..., '1630', 2453584.5, '1800']],
      dtype=object)

In [164]:
# newDiscoverTime = X.loc[:, "DISCOVERY_TIME": "DISCOVERY_TIME"].apply(convertJulianTimeToCategory, axis=1)
# newContTime = X.loc[:, "CONT_TIME": "CONT_TIME"].apply(convertJulianTimeToCategory, axis=1)
# X.loc[:, "DISCOVERY_TIME": "DISCOVERY_TIME"] = newDiscoverTime
# X.loc[:, "CONT_TIME": "CONT_TIME"] = newContTime
# X = X.values

In [165]:
X = encodeCategoricalData(X, 0)
X = encodeCategoricalData(X, 4)
# X = encodeCategoricalData(X, 7)
# X = encodeCategoricalData(X, 8)
X = encodeHotEncoder(X, 0)
X = encodeHotEncoder(X, 4)
# X = encodeHotEncoder(X, 7)
# X = encodeHotEncoder(X, 8)

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [176]:
createNaiveBayesModel(X_train, y_train, X_test, y_test)

Training set Score:  0.03
Testing set Score:  0.021333333333333333


In [182]:
createArtificialNeuralNetwork(X_train, y_train)

Epoch 1/200
3500/3500 [==============================] - 2s 609us/step - loss: 13.8084 - acc: 0.5143
Epoch 2/200
3500/3500 [==============================] - 0s 29us/step - loss: 13.2997 - acc: 0.5143
Epoch 3/200
3500/3500 [==============================] - 0s 28us/step - loss: 13.2997 - acc: 0.5143
Epoch 4/200
3500/3500 [==============================] - 0s 28us/step - loss: 13.2997 - acc: 0.5143
Epoch 5/200
3500/3500 [==============================] - 0s 29us/step - loss: 13.2997 - acc: 0.5143
Epoch 6/200
3500/3500 [==============================] - 0s 28us/step - loss: 13.2997 - acc: 0.5143
Epoch 7/200
3500/3500 [==============================] - 0s 27us/step - loss: 13.2997 - acc: 0.5143
Epoch 8/200
3500/3500 [==============================] - 0s 28us/step - loss: 13.2997 - acc: 0.5143
Epoch 9/200
3500/3500 [==============================] - 0s 27us/step - loss: 13.2997 - acc: 0.5143
Epoch 10/200
3500/3500 [==============================] - 0s 27us/step - loss: 13.2997 - acc: 0.514

3500/3500 [==============================] - 0s 30us/step - loss: 13.2997 - acc: 0.5143
Epoch 83/200
3500/3500 [==============================] - 0s 31us/step - loss: 13.2997 - acc: 0.5143
Epoch 84/200
3500/3500 [==============================] - 0s 29us/step - loss: 13.2997 - acc: 0.5143
Epoch 85/200
3500/3500 [==============================] - 0s 26us/step - loss: 13.2997 - acc: 0.5143
Epoch 86/200
3500/3500 [==============================] - 0s 27us/step - loss: 13.2997 - acc: 0.5143
Epoch 87/200
3500/3500 [==============================] - 0s 26us/step - loss: 13.2997 - acc: 0.5143
Epoch 88/200
3500/3500 [==============================] - 0s 25us/step - loss: 13.2997 - acc: 0.5143
Epoch 89/200
3500/3500 [==============================] - 0s 27us/step - loss: 13.2997 - acc: 0.5143
Epoch 90/200
3500/3500 [==============================] - 0s 27us/step - loss: 13.2997 - acc: 0.5143
Epoch 91/200
3500/3500 [==============================] - 0s 26us/step - loss: 13.2997 - acc: 0.5143
Epo

3500/3500 [==============================] - 0s 25us/step - loss: 13.2997 - acc: 0.5143
Epoch 163/200
3500/3500 [==============================] - 0s 27us/step - loss: 13.2997 - acc: 0.5143
Epoch 164/200
3500/3500 [==============================] - 0s 26us/step - loss: 13.2997 - acc: 0.5143
Epoch 165/200
3500/3500 [==============================] - 0s 26us/step - loss: 13.2997 - acc: 0.5143
Epoch 166/200
3500/3500 [==============================] - 0s 25us/step - loss: 13.2997 - acc: 0.5143
Epoch 167/200
3500/3500 [==============================] - 0s 27us/step - loss: 13.2997 - acc: 0.5143
Epoch 168/200
3500/3500 [==============================] - 0s 27us/step - loss: 13.2997 - acc: 0.5143
Epoch 169/200
3500/3500 [==============================] - 0s 29us/step - loss: 13.2997 - acc: 0.5143
Epoch 170/200
3500/3500 [==============================] - 0s 27us/step - loss: 13.2997 - acc: 0.5143
Epoch 171/200
3500/3500 [==============================] - 0s 28us/step - loss: 13.2997 - acc: 0

In [180]:
creatingRandomForestPredictor(X_train, y_train, X_test, y_test)


Random Forest Classifier Section
---------------------------------
Training set Score:  0.8211428571428572
Testing set Score:  0.6326666666666667


In [82]:
# create RNN

# initialize the RNN
classifier = Sequential()

classifier.add(Embedding(input_dim=1,input_length=50, output_dim=1, trainable=False, mask_zero=True))

classifier.add(Masking(mask_value=0.0))

classifier.add(LSTM(64, return_sequences=False, dropout=0.1, recurrent_dropout=0.1))

classifier.add(Dense(64, activation="relu"))

classifier.add(Dropout(0.5))

classifier.add(Dense(1, activation="softmax"))

classifier.compile(optimizer="adam", loss="mean_squared_error", metrics=["accuracy"])

classifier.fit(X_train, y_train, batch_size=100, epochs=20)

Epoch 1/20


InvalidArgumentError: indices[0,3] = 1 is not in [0, 1)
	 [[{{node embedding_16/embedding_lookup}} = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_16/embeddings/read, embedding_16/Cast, embedding_16/embedding_lookup/axis)]]

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)